In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [2]:
true = pd.read_csv('../raw_data/True.csv')

fake = pd.read_csv('../raw_data/Fake.csv')

In [3]:
true.head()
#fake['text'][0]
#fake.isna().sum()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
#from nltk.tokenize import word_tokenize
#test = fake['text'][0]
#print(word_tokenize(test))

In [5]:
#test.count('@')

In [6]:
#fake['tokenized_text'] = fake.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
#fake.head(2)

In [7]:
import re
# count twitter references in fake['text']
twitter_at = re.compile(r'@([A-Za-z0-9_]+)')
count = 0
for text in fake['text']:
     count += len(re.findall(twitter_at, text))
count

27886

In [8]:
# all_texts_tokenized = []
# for text in fake['text']:
#     all_texts_tokenized.append(word_tokenize(text))
# twit = 0
# for i in range(len(all_texts_tokenized)):
#     for token in all_texts_tokenized[i]:
#         if token == '@':
#             twit += 1
# twit

In [9]:
# count twitter references in true['text']
twitter_at = re.compile(r'@([A-Za-z0-9_]+)')
count = 0
for text in true['text']:
     count += len(re.findall(twitter_at, text))
count

803

In [10]:
true.drop(columns = ['subject','date'], inplace = True)

In [11]:
fake.drop(columns = ['subject','date'], inplace = True)

In [12]:
true['score'] = 1
fake['score'] = 0

In [13]:
data = pd.concat([true,fake],ignore_index=True)

In [14]:
data_title_text = data.copy()

In [15]:
data_title_text['title_text'] = data_title_text['title'] + data_title_text['text']

In [16]:
data_title_text = data_title_text[['title_text', 'score']]

In [17]:
#lower case
data['text'] = data['text'].str.lower()

data['title'] = data['title'].str.lower()

In [18]:
#get rid of numbers
def no_num(text):
    text= ''.join(word for word in text if not word.isdigit())
    return text

data['text']=data['text'].apply(no_num)

In [19]:
#data['text']=data['text'].apply(clean)

In [20]:
#data['text']

In [21]:
#import string 
#def remove_punctuations(text):
      #return text.translate(None,string.punctuation)

data['text'] = data['text'].str.replace('[^\w\s]','')


In [22]:
data['title'] = data['title'].str.replace('[^\w\s]','')

In [23]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
data['title'] = data['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [24]:
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [40]:
data_common_true=true.copy()
data_common_true.drop(columns = ['score'], inplace = True)
data_common_true['title_text'] = data_common_true['title'] + data_common_true['text']
data_common_true.drop(columns = ['title','text'], inplace = True)

data_common_fake=fake.copy()
data_common_fake.drop(columns = ['score'], inplace = True)
data_common_fake['title_text'] = data_common_fake['title'] + data_common_fake['text']
data_common_fake.drop(columns = ['title','text'], inplace = True)

# lower case
data_common_true['title_text'] = data_common_true['title_text'].str.lower()
data_common_fake['title_text'] = data_common_fake['title_text'].str.lower()
# get rid of numbers
data_common_true['title_text'].apply(no_num)
data_common_fake['title_text'].apply(no_num)
# no punctuation
data_common_true['title_text'] = data_common_true['title_text'].str.replace('[^\w\s]','')
data_common_fake['title_text'] = data_common_fake['title_text'].str.replace('[^\w\s]','')
#stop words
data_common_true['title_text'] = data_common_true['title_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
data_common_fake['title_text'] = data_common_fake['title_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [26]:
# #tokenize each row
# data_common_true['title_text_token'] = data_title_text.apply(lambda row: nltk.word_tokenize(row['title_text']), axis=1)
# data_common_fake['title_text_token'] = data_title_text.apply(lambda row: nltk.word_tokenize(row['title_text']), axis=1)


In [27]:
# textify_true = data_common_true['title_text_token'].astype(str).values.tolist()
# textify_fake = data_common_fake['title_text_token'].astype(str).values.tolist()

In [28]:
# wordcount_true = {}
# for word in textify_true:

#     if word not in wordcount_true:
#         wordcount_true[word] = 1
#     else:
#         wordcount_true[word] += 1


In [29]:
# # Print most common word
# n_print = int(input("How many most common words to print: "))
# print("\nOK. The {} most common words are as follows\n".format(n_print))
# word_counter = collections.Counter(wordcount_true)
# for word, count in word_counter.most_common(n_print):
#     print(word, ": ", count)


In [30]:
# lst_true = word_counter.most_common(n_print)
# df_true = pd.DataFrame(lst_true, columns = ['Word', 'Count'])
# df_true.plot.bar(x='Word',y='Count')

In [31]:
# wordcount_fake = {}
# for word in textify_fake:

#     if word not in wordcount_fake:
#         wordcount_fake[word] = 1
#     else:
#         wordcount_fake[word] += 1



In [32]:
# # Print most common word
# n_print = int(input("How many most common words to print: "))
# print("\nOK. The {} most common words are as follows\n".format(n_print))
# word_counter = collections.Counter(wordcount_fake)
# for word, count in word_counter.most_common(n_print):
#     print(word, ": ", count)

In [33]:
# lst_fake = word_counter.most_common(n_print)
# df_fake = pd.DataFrame(lst_fake, columns = ['Word', 'Count'])
# df_fake.plot.bar(x='Word',y='Count')

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
def get_top_words(text, n=None):
    vec = CountVectorizer().fit(text)
    bag_of_words = vec.transform(text)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

list_fake = get_top_words(data_common_fake['title_text'], 25)
list_true = get_top_words(data_common_true['title_text'], 25)


[('trump', 80034),
 ('said', 31196),
 ('people', 26362),
 ('president', 26204),
 ('would', 23664),
 ('one', 23287),
 ('us', 22970),
 ('obama', 19384),
 ('clinton', 18858),
 ('like', 17941),
 ('donald', 16787),
 ('also', 15247),
 ('new', 14988),
 ('hillary', 14955),
 ('news', 14910),
 ('even', 13799),
 ('white', 13559),
 ('time', 12927),
 ('state', 12866),
 ('media', 11573),
 ('via', 11165),
 ('get', 11051),
 ('house', 10997),
 ('campaign', 10978),
 ('america', 10957)]

In [43]:
import plotly.graph_objects as go
new_list_words = [ seq[0] for seq in list_fake ]
new_list_values = [ seq[1] for seq in list_fake ]

fig = go.Figure()
fig.add_trace(go.Bar(y=new_list_values, 
                         x=new_list_words,
))

fig.show()

In [44]:
new_list_words = [ seq[0] for seq in list_true ]
new_list_values = [ seq[1] for seq in list_true ]

fig = go.Figure()
fig.add_trace(go.Bar(y=new_list_values, 
                         x=new_list_words,
))

fig.show()